In [5]:
import requests
import json
from datetime import datetime
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium


In [6]:
# API keys
OPENWEATHERMAP_API_KEY = 'f3302c23a79dd1a5594fee0f2e7ce39b'
OPENEI_API_KEY = 'bt75TVtSexbJH42JfvxHCnep6xkeBzqK8BaQISYR'
OPENROUTESERVICE_API_KEY = 'your_openrouteservice_api_key'
FLIGHTAWARE_API_KEY = 'your_flightaware_api_key'
MARINETRAFFIC_API_KEY = 'your_marinetraffic_api_key'

In [7]:
# Supplier locations (coordinates)

import geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my-app")

SUPPLIER_LOCATIONS = []

data= [
    ("AGC Automotive Americas Co.", "Illinois"),
    ("VALEO ENGINE COOLING INC. ATTN. FRANCINE MACADAM", "Michigan"),
    ("TEKLAS USA INC", "Georgia"),
    ("DUS Operating Inc.", "Alabama"),
    ("GEDIA Georgia, LLC", "Georgia"),
    ("Eissmann Automotive Detroit Development LLC", "Michigan"),
    ("KIRCHHOFF Automotive Atlanta Inc.", "Georgia"),
    ("Carcoustics USA South, Inc.", "Georgia"),
    ("VALEO North America Inc.", "Tennessee"),
    ("Gestamp Chattanooga, LLC", "Tennessee"),
    ("Nemak USA Inc.", "Wisconsin"),
    ("Magna Mirrors of America - SPD Service Parts Division", "Michigan"),
    ("Autoneum North America, Incorp", "Indiana"),
    ("IAC Cottondale Alabama USA", "Alabama"),
    ("Aludyne North America LLC.", "Wisconsin"),
    ("Fujikura Automotive America LLC", "Texas"),
    ("MDA US LLC", "Tennessee"),
    ("GEDIA Michigan, Inc.", "Michigan"),
    ("Antolin St Clair, LLC", "Michigan"),
    ("Aurea Software, Inc", "Texas"),
    ("MOBIS US Alabama, LLC", "Alabama"),
    ("Auria Old Fort, LLC", "North Carolina"),
    ("Boysen USA Alabama LLC", "Alabama"),
    ("ZF Transmissions Gray Court, LLC", "South Carolina")
]

data_test= [
    ("AGC Automotive Americas Co.", "Illinois"),
    ("VALEO ENGINE COOLING INC. ATTN. FRANCINE MACADAM", "Michigan"),
    ("TEKLAS USA INC", "Georgia"),
    ("DUS Operating Inc.", "Alabama"),
]

for i, (name, state) in enumerate(data, start=1):
    location = geolocator.geocode(state, exactly_one=True)
    if location is None:
        print(f"No location found for {name} in {state}")
    else:
        coordinates = [location.latitude, location.longitude]
        supplier = {
            "id": f"supplier_{i}",
            "name": name,
            "state": state,
            "coordinates": coordinates
        }
        SUPPLIER_LOCATIONS.append(supplier)

print(SUPPLIER_LOCATIONS)


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Illinois&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [3]:
import pandas as pd
from geopy.geocoders import Nominatim

# create a geolocator object
geolocator = Nominatim(user_agent="my_application")

data= [
    ("AGC Automotive Americas Co.", "Illinois"),
    ("VALEO ENGINE COOLING INC. ATTN. FRANCINE MACADAM", "Michigan"),
    ("TEKLAS USA INC", "Georgia"),
    ("DUS Operating Inc.", "Alabama"),
    ("GEDIA Georgia, LLC", "Georgia"),
    ("Eissmann Automotive Detroit Development LLC", "Michigan"),
    ("KIRCHHOFF Automotive Atlanta Inc.", "Georgia"),
    ("Carcoustics USA South, Inc.", "Georgia"),
    ("VALEO North America Inc.", "Tennessee"),
    ("Gestamp Chattanooga, LLC", "Tennessee"),
    ("Nemak USA Inc.", "Wisconsin"),
    ("Magna Mirrors of America - SPD Service Parts Division", "Michigan"),
    ("Autoneum North America, Incorp", "Indiana"),
    ("IAC Cottondale Alabama USA", "Alabama"),
    ("Aludyne North America LLC.", "Wisconsin"),
    ("Fujikura Automotive America LLC", "Texas"),
    ("MDA US LLC", "Tennessee"),
    ("GEDIA Michigan, Inc.", "Michigan"),
    ("Antolin St Clair, LLC", "Michigan"),
    ("Aurea Software, Inc", "Texas"),
    ("MOBIS US Alabama, LLC", "Alabama"),
    ("Auria Old Fort, LLC", "North Carolina"),
    ("Boysen USA Alabama LLC", "Alabama"),
    ("ZF Transmissions Gray Court, LLC", "South Carolina")
]

# create an empty DataFrame to store the supplier locations
columns = ["id", "name", "state", "latitude", "longitude"]
supplier_locations = pd.DataFrame(columns=columns)

# loop through the data and geocode the state for each supplier
for i, (name, state) in enumerate(data, start=1):
    location = geolocator.geocode(state, exactly_one=True)
    if location is None:
        print(f"No location found for {name} in {state}")
    else:
        # extract the latitude and longitude from the location object
        latitude, longitude = location.latitude, location.longitude
        # create a dictionary for the supplier and append it to the DataFrame
        supplier_dict = {"id": f"supplier_{i}",
                         "name": name,
                         "state": state,
                         "latitude": latitude,
                         "longitude": longitude}
        supplier_locations = supplier_locations.append(supplier_dict, ignore_index=True)

# print the supplier locations DataFrame
print(supplier_locations)


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Illinois&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [5]:
# Fetch weather data
def get_weather_data(coordinates):
    url = f'https://api.openweathermap.org/data/2.5/weather?lat={coordinates[0]}&lon={coordinates[1]}&appid={OPENWEATHERMAP_API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None
        
# Fetch power outage data
def get_power_outage_data():
    url = f'https://utilityapi.com/api/v2/outages?access_token={OPENEI_API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Retrieve power outage data
def current_condition(coordinates):
    latitude = coordinates[0]
    longitude = coordinates[1]

    # Base URL for National Weather Service API
    base_url = "https://api.weather.gov"

    # Get grid information for the given latitude and longitude
    grid_url = f"{base_url}/points/{latitude},{longitude}"
    try:
        grid_response = requests.get(grid_url, headers={'User-Agent': 'weather_app'})
        grid_response.raise_for_status()
        grid_data = grid_response.json()

        # Extract the forecast and warning URLs
        forecast_url = grid_data['properties']['forecast']
        warnings_url = grid_data['properties']['forecastZone'].replace('forecastzones', 'alerts/active/zone')

        # Get the forecast data
        forecast_response = requests.get(forecast_url, headers={'User-Agent': 'weather_app'})
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()

        # Print the current weather condition
        current_conditions = forecast_data['properties']['periods'][0]
        return current_conditions['shortForecast']

    except requests.exceptions.HTTPError as error:
        print(f"Error: {error}")
        return None
    except requests.exceptions.RequestException as error:
        print(f"Error: {error}")
        return None


In [6]:
# Create an empty DataFrame to store the data
df = pd.DataFrame()

# Loop through each supplier location
for supplier in SUPPLIER_LOCATIONS:

    # Get current weather condition
    weather_data = current_condition(supplier['coordinates'])

    # Get supplier information
    supplier_id = supplier['id']
    supplier_name = supplier['name']
    state = supplier['state']
    latitude = supplier['coordinates'][0]
    longitude = supplier['coordinates'][1]

    # Create a dictionary to store the data
    data = {'Supplier ID': supplier_id,
            'Supplier Name': supplier_name,
            'State': state,
            'Latitude': latitude,
            'Longitude': longitude,
            'Weather Data': weather_data}

    # Append the dictionary to the DataFrame
    df = df.append(data, ignore_index=True)

# Save the DataFrame to a CSV file
df.to_csv('supplier_data.csv', index=False)
    

/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


Error: 500 Server Error: Internal Server Error for url: https://api.weather.gov/gridpoints/GRR/72,80/forecast


/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

Error: 500 Server Error: Internal Server Error for url: https://api.weather.gov/gridpoints/IND/56,94/forecast


/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

Error: 500 Server Error: Internal Server Error for url: https://api.weather.gov/gridpoints/RAH/61,51/forecast


/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_2629/1737445879.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)


In [27]:
# Calculate risk score based on weather and power outage
def calculate_risk_score(weather_data, power_outage_data):
    risk_score = 0

    # Weather risk
    if weather_data and weather_data.get("weather") and weather_data["weather"][0]["main"] in ["Thunderstorm", "Rain", "Snow"]:
        risk_score += 1

    # Power outage risk
    if power_outage_data and power_outage_data.get("outages") and len(power_outage_data["outages"]) > 0:
        risk_score += 1

    return risk_score

In [ ]:
def analyze_data():
    disruption_risk = []

    # Loop through supplier locations
    for supplier in SUPPLIER_LOCATIONS:
        weather_data = get_weather_data(supplier['coordinates'])
        power_outage_data = get_power_outage_data()

        # Calculate disruption risk based on weather and power outage
        risk_score = calculate_risk_score(weather_data, power_outage_data)
        disruption_risk.append({
            "supplier_id": supplier["id"],
            "supplier_name": supplier["name"],
            "coordinates": supplier["coordinates"],
            "risk_score": risk_score
        })

    return disruption_risk

In [32]:
# Display results in a dashboard or report
def display_results(disruption_risk):
    # Create a GeoDataFrame
    geometry = [Point(xy) for xy in [x["coordinates"] for x in disruption_risk]]
    gdf = gpd.GeoDataFrame(disruption_risk, geometry=geometry)

    # Generate an interactive map
    m = folium.Map(location=[38.8951100, -77.0363700], zoom_start=4)

    # Add supplier markers with risk score
    for index, row in gdf.iterrows():
        color = "green" if row["risk_score"] == 0 else "orange" if row["risk_score"] == 1 else "red"
        folium.Marker(location=[row["coordinates"][0], row["coordinates"][1]], popup=f'Supplier ID: {row["supplier_id"]}<br>supplier: {row["supplier_name"]}<br>Risk Score: {row["risk_score"]}', icon=folium.Icon(color=color)).add_to(m)

    return m

In [35]:
from IPython.display import display

def main():
    disruption_risk = analyze_data()
    dashboard = display_results(disruption_risk)
    dashboard.save("sample.html")
    display(dashboard)

if __name__ == "__main__":
    main()


In [1]:
import requests
import json

# Replace these with the latitude and longitude of your location
latitude = 	33.543682
longitude = -86.779633

# Base URL for National Weather Service API
base_url = "https://api.weather.gov"

# Get grid information for the given latitude and longitude
grid_url = f"{base_url}/points/{latitude},{longitude}"
grid_response = requests.get(grid_url, headers={'User-Agent': 'weather_app'})
#grid_response = requests.get(grid_url) 
grid_data = grid_response.json()

# Extract the forecast and warning URLs
forecast_url = grid_data['properties']['forecast']
warnings_url = grid_data['properties']['forecastZone'].replace('forecastzones', 'alerts/active/zone')

# Get the forecast data
forecast_response = requests.get(forecast_url, headers={'User-Agent': 'weather_app'})
forecast_data = forecast_response.json()

# Print the current weather condition
current_conditions = forecast_data['properties']['periods'][0]
print(f"Current Weather Condition: {current_conditions['shortForecast']}")

# Get weather warnings
warnings_response = requests.get(warnings_url, headers={'User-Agent': 'weather_app'})
warnings_data = warnings_response.json()

# Check for issues in the API response and print warnings
if warnings_response.status_code != 200 or 'features' not in warnings_data:
    print("Error fetching weather warnings. Please try again later.")
else:
    if len(warnings_data['features']) == 0:
        print("No weather warnings at this time.")
    else:
        for warning in warnings_data['features']:
            print(f"Weather Warning: {warning['properties']['event']}")


Current Weather Condition: Slight Chance Showers And Thunderstorms
Error fetching weather warnings. Please try again later.


In [3]:
grid_url, forecast_url

('https://api.weather.gov/points/33.543682,-86.779633',
 'https://api.weather.gov/gridpoints/BMX/60,86/forecast')

In [11]:
# define a function to split the text and return a list of words
def split_text(text):
    return text.split()

In [13]:
# drop rows with missing values
supplier_df.dropna(subset=['Weather Data'], inplace=True)

# define a function to split the text and return a list of words
def split_text(text):
    return text.split()

# apply the function to the "Weather Data" column and create a new column
supplier_df['Weather List'] = supplier_df['Weather Data'].apply(split_text)

In [16]:
supplier_df.head()

,Supplier ID,Supplier Name,State,Latitude,Longitude,Weather Data,Weather List
0,supplier_1,AGC Automotive Americas Co.,Illinois,40.079661,-89.433729,Slight Chance Showers And Thunderstorms then M...,"[Slight, Chance, Showers, And, Thunderstorms, ..."
2,supplier_3,TEKLAS USA INC,Georgia,32.329381,-83.113737,Patchy Fog then Slight Chance Rain Showers,"[Patchy, Fog, then, Slight, Chance, Rain, Show..."
3,supplier_4,DUS Operating Inc.,Alabama,33.258882,-86.829534,Patchy Fog then Slight Chance Showers And Thun...,"[Patchy, Fog, then, Slight, Chance, Showers, A..."
4,supplier_5,"GEDIA Georgia, LLC",Georgia,32.329381,-83.113737,Patchy Fog then Slight Chance Rain Showers,"[Patchy, Fog, then, Slight, Chance, Rain, Show..."
5,supplier_6,Eissmann Automotive Detroit Development LLC,Michigan,43.621195,-84.682435,Chance Light Rain,"[Chance, Light, Rain]"


In [23]:
# define a function to calculate the risk score based on weather conditions
def calculate_risk(weather_list):
    risk_score = 0
    for weather in weather_list:
        if weather in ["Thunderstorm","Thunderstorms", "Rain", "Snow"]:
            risk_score = 1
            break
    return risk_score

# apply the function to the "Weather List" column and create a new column
supplier_df['Weather Risk'] = supplier_df['Weather List'].apply(calculate_risk)


In [24]:
supplier_df.head()

,Supplier ID,Supplier Name,State,Latitude,Longitude,Weather Data,Weather List,Weather Risk
0,supplier_1,AGC Automotive Americas Co.,Illinois,40.079661,-89.433729,Slight Chance Showers And Thunderstorms then M...,"[Slight, Chance, Showers, And, Thunderstorms, ...",1
2,supplier_3,TEKLAS USA INC,Georgia,32.329381,-83.113737,Patchy Fog then Slight Chance Rain Showers,"[Patchy, Fog, then, Slight, Chance, Rain, Show...",1
3,supplier_4,DUS Operating Inc.,Alabama,33.258882,-86.829534,Patchy Fog then Slight Chance Showers And Thun...,"[Patchy, Fog, then, Slight, Chance, Showers, A...",1
4,supplier_5,"GEDIA Georgia, LLC",Georgia,32.329381,-83.113737,Patchy Fog then Slight Chance Rain Showers,"[Patchy, Fog, then, Slight, Chance, Rain, Show...",1
5,supplier_6,Eissmann Automotive Detroit Development LLC,Michigan,43.621195,-84.682435,Chance Light Rain,"[Chance, Light, Rain]",1


In [26]:
# Save the DataFrame to a CSV file
supplier_df.to_csv('supplier_data.csv', index=False)